In [1]:
import numpy as np
import healpy as hp
import pandas as pd
import os
output_dir = "mask4"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

for k in range(3):
    f = 'data/quasar_sample_' + str(k+1) + '.csv'
    fd0 = pd.read_csv(f)

    # Apply Galactic latitude mask
    mask = (fd0['b'] >= 40) | (fd0['b'] <= -60)
    fd = fd0[mask].reset_index(drop=True)

    l = fd['l'].to_numpy()
    b = fd['b'].to_numpy()

    l_rad = np.radians(l)
    b_rad = np.radians(b)

    # angular distance from (l=0, b=0)
    cos_theta = np.cos(b_rad) * np.cos(l_rad)
    theta_rad = np.arccos(cos_theta)
    theta_deg = np.degrees(theta_rad)

    # Apply circular mask of radius 68.6° which correspons to 4 sr solid angle centered l = 0 and b = 0  
    mask1 = theta_deg > 68.6
    fd1 = fd[mask1].reset_index(drop=True)
    
    r = fd1['r'].to_numpy()    
    th = fd1['theta'].to_numpy()
    ph = fd1['phi'].to_numpy()

    Nside= 64                 
    Npix=hp.nside2npix(Nside)

   
    m=np.zeros(Npix, dtype= int)
    for i in range(len(r)):
        px=hp.ang2pix(Nside, th[i], ph[i])
        m [px]+=1

    P=[]
    for j in range(Npix):
        if(m[j]>0):
            P.append(j)
    p = np.array(P)
    df1 = pd.DataFrame(data=p)
    f1 = 'mask4/non_zero_pix_id_' + str(k+1) + '.dat'

    df1.to_csv(f1,sep='\t',header = None,index = False)
            
    op = 'mask4/masked_sample_' + str(k+1) + '.dat'
    df_masked = pd.DataFrame({'r': r,'theta': th,'phi': ph})
    df_masked.to_csv(op,sep = '\t',header = None,index = False)
